In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Iris.csv to Iris.csv


In [27]:
import pandas as pd

df = pd.read_csv('/content/SamplesFile_b_4C2D800Linear.csv')
df = df.drop(['Cluster'], axis = 1)
df.head(20)

,Mean,Covariance
0,-9.769318,-9.106956
1,13.638444,9.219981
2,-10.065482,8.641450
3,9.934749,-15.189142
4,-10.859542,-9.019489
5,9.042867,11.274845
6,-8.890605,10.300685
7,12.035744,-7.540852
8,-8.830689,-9.626402
9,13.613286,13.136570


In [28]:
print(df.shape)

(800, 2)


In [29]:
#import matplotlib.pyplot as plt
#df.plot(kind="scatter", x="SepalLengthCm", y="SepalWidthCm")
#plt.show()

In [30]:

import numpy as np
import scipy.spatial
import math

from collections import Counter


MinPts = 80
Emax = 6.0
Emin = 4.0
visited = [False] * 800
outlierLs = []

def regionQuery(index, df, eps):
  '''
  df - dataset given.
  index - current data point under consideration.
  eps - value of the epsilon-neighborhood of the datapoint
  '''
  w, x = df.iloc[index]['Mean']  ,  df.iloc[index]['Covariance']; #, df.iloc[index]['PetalLengthCm'], df.iloc[index]['PetalWidthCm']

  temp =  df[(np.abs(w - df['Mean']) <= eps) & (np.abs(x - df['Covariance']) <= eps) & (df.index != index)];

  return temp.index

def cardinality(list):
  return len(list)

# finds distance between 'point' and 'i' (indexes of corresponding points from the dataframe)
def compute(point,i,df):
   w, x = df.iloc[i]['Mean'],  df.iloc[i]['Covariance'];#, df.iloc[i]['PetalLengthCm'], df.iloc[i]['PetalWidthCm'];
   a, b = df.iloc[point]['Mean']  ,  df.iloc[point]['Covariance'];#, df.iloc[point]['PetalLengthCm'], df.iloc[point]['PetalWidthCm'];
   distance=math.sqrt(pow(w-a,2)+pow(x-b,2));
   return distance;

# the following two functions find 'mu' (fuzzy point membership value)
def computemu(dmin, df):
  mu=(dmin-Emax)/(Emin-Emax);
  return mu;


def computeU(indexofdp ,df, corepointlistofcurrentcluster):
  dlist=[]
  for cp in corepointlistofcurrentcluster:
    d1=compute(cp,indexofdp,df);
    dlist.append(d1);
  dmin=min(dlist);
  if dmin>Emax:
    return 0;
  elif dmin<Emin:
    return 1;
  else:
    mu=computemu(dmin, df) #formula to compute uxc
    return mu;



# main function that performs the DBSCAN algorithm
def SF_DBSCAN(dataset):

  currentCluster = []
  clusters = []
  corepoints=[]
  muLs = []

  # adding a new column in
  dataset['currentcluster'] = 0
  dictClusters = {}
  for dataPoint in range(len(dataset)):
    dictClusters[dataPoint] = []

  for dataPoint in range(len(dataset)):
    if visited[dataPoint]==False:          # FOR ALL UNVISITED DATAPOINTS
      visited[dataPoint] = True            # MARK P AS VISITED

      dataPointMin = regionQuery(dataPoint, dataset, Emin)      # pMin = regionQuery(p, eMin) - GETTING ALL POINTS WITHIN EMIN DISTANCE OF POINT
      if cardinality(dataPointMin) < MinPts:                   # if cardinality(pMin) < MinPts
        outlierLs.append(dataPoint)                             # THEN LABEL P AS OUTLIER AND CONTINUTE TO NEXT DATA POINT
        dataset["currentcluster"][dataPoint] = -1
        dictClusters[dataPoint].append((-1,-1))
        visited[dataPoint] = True
        continue

      else:                                       # ELSE CREATE A NEW CLUSTER AND PERFORM "EXPAND CLUSTER"
        currentCluster = []
        corepoints = []
        # cls = expandCluster(dataPoint, dataPointMin, currentCluster, dataset, corepoints)
        currentCluster.append((dataPoint,1))                 # ADD P TO C AS CORE POINT (WITH MEMBERSHIP 1)
        dataset['currentcluster'][dataPoint] = len(clusters)
        dictClusters[dataPoint].append((len(clusters),1))
        corepoints.append(dataPoint)

        temp = regionQuery(dataPoint, dataset, Emax)            # pMinMax = regionQuery(p, emin) \ pMin
        dataPointMinMax = set(temp) - set(dataPointMin)
        fuzzyBorder = list(dataPointMinMax)                     # fuzzyBorder = pMinMax

        for neighbor in dataPointMin:                           # for s belongs to pMin (we will mark 's' as visited only when we are adding to any cluster)
          neighborMin = regionQuery(neighbor, dataset, Emin)      # sMin =regionQuery(s, emin)

          if cardinality(neighborMin) >= MinPts:                     # if cardinality(sMin) >= MinPts
            dataPointMin = list(set(dataPointMin).union(set(neighborMin)))    # pMin = pMin U sMin
            temp = regionQuery(neighbor, dataset, Emax)                       # sMinMax = regionQuery(s, emax) \ sMin
            neighborMinMax = set(temp) - set(neighborMin)
            fuzzyBorder = list(set(fuzzyBorder).union(neighborMinMax))        # fuzzyBorder = fuzzyBorder U sMinMax
            if visited[neighbor] == False:
              currentCluster.append((neighbor,1))                     # add s to C as core
              dataset['currentcluster'][dataPoint] = len(clusters)
              dictClusters[neighbor].append((len(clusters),1))
              corepoints.append(neighbor);
              visited[neighbor] = True                                # mark s as visited

          else:                                                      # else fuzzyBorder = fuzzyBorder U {s} (adding s to fuzzyBorder)
            if neighbor not in fuzzyBorder:
              fuzzyBorder.append(neighbor);


        clusterFirstPts = [pt[0] for pt in currentCluster]
        fuzzyBorder = list(set(fuzzyBorder) - set(clusterFirstPts))     # fuzzyBorder = fuzzyBorder \ currenCluster

        for x in fuzzyBorder:                      # FOR x belongs to fuzzyBorder
          if visited[x] == False:                   # add x to C as border object with membership µxc where c is the nearest core of C
            mu1=computeU(x, df, corepoints)               # compute mu from corepoints
            muLs.append((x,mu1))
            if mu1!=0:
              t=(x,mu1);
              currentCluster.append(t);
              visited[x] = True                            # mark x (earlier s) as visited
              dataset['currentcluster'][x] = len(clusters)
              dictClusters[x].append((len(clusters),mu1))

        print("hello:", currentCluster)
        print(len(currentCluster))
        clusters.append(currentCluster)


  for i in muLs:
    if i[1] == 0:
      outlierLs.append(i[0])

  print(len(outlierLs))
  print(outlierLs)
  print(corepoints)
  return (clusters, dataset, dictClusters)




In [31]:
(_, _, dictClusters) = SF_DBSCAN(df)

#print(a)
#print(df_new.shape)

<ipython-input-30-2feca714abe9>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['currentcluster'][dataPoint] = len(clusters)
<ipython-input-30-2feca714abe9>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['currentcluster'][dataPoint] = len(clusters)
<ipython-input-30-2feca714abe9>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['currentcluster'][x] = len(clusters)


hello: [(0, 1), (8, 1), (12, 1), (20, 1), (24, 1), (28, 1), (32, 1), (36, 1), (40, 1), (44, 1), (48, 1), (52, 1), (56, 1), (60, 1), (64, 1), (68, 1), (72, 1), (76, 1), (80, 1), (84, 1), (88, 1), (92, 1), (96, 1), (100, 1), (104, 1), (108, 1), (112, 1), (116, 1), (120, 1), (124, 1), (128, 1), (132, 1), (136, 1), (140, 1), (144, 1), (148, 1), (152, 1), (156, 1), (160, 1), (164, 1), (168, 1), (172, 1), (176, 1), (180, 1), (184, 1), (188, 1), (192, 1), (196, 1), (200, 1), (204, 1), (208, 1), (212, 1), (216, 1), (220, 1), (224, 1), (228, 1), (232, 1), (236, 1), (240, 1), (244, 1), (248, 1), (252, 1), (256, 1), (260, 1), (264, 1), (268, 1), (272, 1), (276, 1), (280, 1), (284, 1), (288, 1), (292, 1), (296, 1), (300, 1), (304, 1), (308, 1), (312, 1), (316, 1), (320, 1), (328, 1), (332, 1), (336, 1), (340, 1), (344, 1), (348, 1), (352, 1), (360, 1), (364, 1), (368, 1), (376, 1), (380, 1), (388, 1), (392, 1), (408, 1), (412, 1), (420, 1), (456, 1), (464, 1), (512, 1), (516, 1), (4, 1), (520, 1),

<ipython-input-30-2feca714abe9>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["currentcluster"][dataPoint] = -1


hello: [(2, 1), (6, 1), (10, 1), (18, 1), (34, 1), (38, 1), (46, 1), (54, 1), (58, 1), (62, 1), (70, 1), (74, 1), (82, 1), (94, 1), (98, 1), (102, 1), (106, 1), (110, 1), (114, 1), (118, 1), (122, 1), (134, 1), (138, 1), (146, 1), (150, 1), (154, 1), (158, 1), (162, 1), (166, 1), (170, 1), (174, 1), (178, 1), (182, 1), (186, 1), (190, 1), (194, 1), (198, 1), (202, 1), (206, 1), (214, 1), (218, 1), (222, 1), (226, 1), (230, 1), (234, 1), (238, 1), (242, 1), (250, 1), (254, 1), (258, 1), (266, 1), (270, 1), (274, 1), (278, 1), (282, 1), (286, 1), (290, 1), (294, 1), (314, 1), (322, 1), (330, 1), (334, 1), (342, 1), (354, 1), (358, 1), (370, 1), (386, 1), (390, 1), (394, 1), (402, 1), (422, 1), (434, 1), (438, 1), (450, 1), (462, 1), (486, 1), (498, 1), (506, 1), (510, 1), (514, 1), (538, 1), (562, 1), (518, 1), (522, 1), (14, 1), (526, 1), (530, 1), (534, 1), (22, 1), (26, 1), (30, 1), (542, 1), (546, 1), (550, 1), (554, 1), (42, 1), (558, 1), (50, 1), (566, 1), (570, 1), (574, 1), (578,

In [32]:
print(dictClusters)

{0: [(0, 1)], 1: [(-1, -1)], 2: [(1, 1)], 3: [(-1, -1)], 4: [(0, 1)], 5: [(-1, -1)], 6: [(1, 1)], 7: [(-1, -1)], 8: [(0, 1)], 9: [(-1, -1)], 10: [(1, 1)], 11: [(-1, -1)], 12: [(0, 1)], 13: [(2, 1)], 14: [(1, 1)], 15: [(-1, -1)], 16: [(0, 1)], 17: [(2, 1)], 18: [(1, 1)], 19: [(-1, -1)], 20: [(0, 1)], 21: [(2, 1)], 22: [(1, 1)], 23: [(-1, -1)], 24: [(0, 1)], 25: [(2, 0.7293330509336182)], 26: [(1, 1)], 27: [(-1, -1)], 28: [(0, 1)], 29: [(2, 1)], 30: [(1, 1)], 31: [(-1, -1)], 32: [(0, 1)], 33: [(2, 1)], 34: [(1, 1)], 35: [(-1, -1)], 36: [(0, 1)], 37: [(2, 1)], 38: [(1, 1)], 39: [(-1, -1)], 40: [(0, 1)], 41: [(2, 1)], 42: [(1, 1)], 43: [(-1, -1)], 44: [(0, 1)], 45: [(2, 1)], 46: [(1, 1)], 47: [(3, 1)], 48: [(0, 1)], 49: [(2, 1)], 50: [(1, 1)], 51: [(3, 0.3756055155819773)], 52: [(0, 1)], 53: [(2, 1)], 54: [(1, 1)], 55: [(3, 1)], 56: [(0, 1)], 57: [(2, 1)], 58: [(1, 1)], 59: [(3, 1)], 60: [(0, 1)], 61: [(2, 1)], 62: [(1, 1)], 63: [(3, 1)], 64: [(0, 1)], 65: [(2, 1)], 66: [(1, 1)], 67: [(3, 

In [33]:
print(dictClusters.get(0))

[]


In [34]:
from sklearn import metrics

(_, _, dictClusters) = SF_DBSCAN(df)

# x_train = df.iloc[:, 2:5].values --for mall dataset
x_train = df[['Mean', 'Covariance']]

labels_pred = []
for datapoint in dictClusters:
  labels_pred.append(dictClusters,get(datapoint))

labels_true = y_label - 1

print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x_train, labels_pred))
print("Adjusted Rand Index:", metrics.adjusted_rand_score(labels_true, labels_pred))
print("Rand Index:", metrics.rand_score(labels_true, labels_pred))
print("CH Index:", metrics.calinski_harabasz_score(x_train, labels_pred))

21
[1, 3, 5, 7, 9, 11, 15, 19, 23, 27, 31, 35, 39, 43, 111, 780, 642, 726, 735, 795, 111]
[]


NameError: ignored